In [5]:
ls /home/jovyan/bin

ape.bin  lexon  lexon-rust



In [33]:
%alias_magic solidity script -p "/home/jovyan/.solcx/solc-v0.6.7 --ast-json -"

Created `%%solidity` as an alias for `%%script /home/jovyan/.solcx/solc-v0.6.7 --ast-json -`.


In [7]:
%alias_magic ape script -p "/home/jovyan/bin/ape.bin -csyntax"

Created `%%ape` as an alias for `%%script /home/jovyan/bin/ape.bin -csyntax`.


In [16]:
%alias_magic lexonv3 script -p "/home/jovyan/bin/lexon"

Created `%%lexonv3` as an alias for `%%script /home/jovyan/bin/lexon`.


In [30]:
%alias_magic lexonv2 script -p "/home/jovyan/bin/lexon-rust <"

Created `%%lexonv2` as an alias for `%%script /home/jovyan/bin/lexon-rust <`.


### Todo
* check with phd-data dir and conf paper for up to date ness

# License Evaluation Contract Representation

In the following notebook the examples of my thesis are accumulated in such a state as to make 

## Natural legal language example

Example:

This is a license agreement for a software evaluation license.

*LICENSOR* - Unnamed Software Company.
*LICENSEE* - Unname University or other Organization.
*SUBLICENSEE* - A student as set force in the appendix. (array of persons)  
*ARBITER* - An arbiter or oracle that decides incase of disputes.  Can be a natural or legal person or a machine. 
Art.  2, 3 and 4 especially are evaluated by the arbiter in case of disputes.  
*ASSET* - An asset to be licensed.  

* Article 1: The  Licensor  grants  the  Licensee  a limited  license  to use  and  evaluate  asset  X  and  grant  sublicenses among group Y, for  use  and  evaluation, in exchange for a licensing fee. This limited license lasts from DD.MM.YYYY (inclusive) to DD.MM.YYYY (exclusive).
* Article 2. (optional) The (Sub)Licensee is commissioned to publish comments  about  the  use  of  the  product, in exchange for an evaluation fee. This allows publication of comments about the evaluation of *ASSET*. Furthermore, this requires the publication of comments about the evaluation of the *ASSET*.
* Article 3. The (Sub)Licensee must not publish comments of the use and evaluation of the Product without the approval  of  the  Licensor; said  approval must be obtained before the publication of comments about the evaluation of *ASSET*. If the Licensee publishes results of the evaluation of the Product without approval from the Licensor, the Licensee has 24 h to remove the material from the time of publication. If the time lapses without a removal of the comments of the evaluation, the license is considered breached.
* Article 4. This license terminates automatically if  the  (Sub)Licensee  breaches  this  Agreement. In case of a breach of main Licensor, this entails breach of all sublicensees. A breach of the Licensing Terms obliges the Licensee to pay a fee to Licensor for Breach of the Licensing Terms.

## Solidity

Soliditiy can be compiled, but for testing its functions, it would have to be copied into remix ide. Also it should be noted that for abbrevation purposes, this contract is not necessarily secure or complete from a usability point of view, such as convenience functions and similar might be missing. This is due to the fact that it would otherwise be too long for publication.

In [34]:
%%solidity
pragma solidity ^0.6.7;
pragma experimental ABIEncoderV2;
//import "github.com/OpenZeppelin/openzeppelin-contracts/contracts/utils/EnumerableMap.sol";

contract LicenseContract {
    //using EnumerableMap for EnumerableMap.UintToAddressMap;
    bytes32 work_hash; string name;
    address payable licensor;
    address licensee;	
    //  EnumerableMap.UintToAddressMap sublicensees;
    address payable arbiter;
    bool breachFeePaid;
    
event Status(LicenseProps);
    
struct LicenseProps {
    int id;
    int licenseFee;
    uint breachFee;
    bool isCommissioned;
    bool publicationIsApproved;
    bool requiresComments;
    uint timeToRemove;
    bool triggeredTimeToRemove;
    bool licenseBreached;}

mapping (uint => LicenseProps) licenses;
uint[] public licensesList;
uint numLicenses;
//EnumerableMap.UintToAddressMap memory _sublicensees,
modifier onlyBy(address _account)
    {require(msg.sender == _account,
          "Sender not authorized.");
        _;}

function newLicense(int _id, int _licenseFee, uint _breachFee, bool _isCommissioned, bool _publicationIsApproved, bool _requiresComments, uint _timeToRemove, bool _triggeredTimeToRemove, bool _licenseBreached) public returns (uint licenseID) {
        licenses[licenseID] = LicenseProps(_id, _licenseFee, _breachFee, _isCommissioned, _publicationIsApproved, _requiresComments, _timeToRemove, _triggeredTimeToRemove, _licenseBreached);
        licenseID = numLicenses++;
        // sublicensees = _sublicensees;
        return licenseID; }

function commissionComments(uint _licenseID) public {
    LicenseProps storage l = licenses[_licenseID];
    l.publicationIsApproved = true;
    l.requiresComments = true;
	l.isCommissioned = true; }

function grantApproval(uint _licenseID) public {
    LicenseProps storage l = licenses[_licenseID];
    l.publicationIsApproved = true; }

function evalPublication(uint _licenseID, bool isPublished) public {
    LicenseProps storage l = licenses[_licenseID];
    if (isPublished) {
        if ((isPublished && !l.isCommissioned || !l.publicationIsApproved) && !l.triggeredTimeToRemove) {
            l.timeToRemove = now;
            l.triggeredTimeToRemove = true;
        } else if (l.triggeredTimeToRemove && now > l.timeToRemove + 1 days ) {
            declareBreach(_licenseID);
        }    }
    emit Status(l); }

function declareRemoved(uint _licenseID) public onlyBy(arbiter) {
    LicenseProps storage l = licenses[_licenseID];
    l.timeToRemove = 0;
    l.triggeredTimeToRemove = false;
}

function declareBreach(uint _licenseID) public onlyBy(arbiter) {
    LicenseProps storage l = licenses[_licenseID];
	l.licenseBreached = true;
	if (!breachFeePaid) {
	    breachFeePaid = false;
	    licensor.transfer(l.breachFee);
	    }     } }


JSON AST:


======= <stdin> =======
{
  "attributes":
  {
    "absolutePath": "<stdin>",
    "exportedSymbols":
    {
      "LicenseContract":
      [
        283
      ]
    }
  },
  "children":
  [
    {
      "attributes":
      {
        "literals":
        [
          "solidity",
          "^",
          "0.6",
          ".7"
        ]
      },
      "id": 1,
      "name": "PragmaDirective",
      "src": "0:23:0"
    },
    {
      "attributes":
      {
        "literals":
        [
          "experimental",
          "ABIEncoderV2"
        ]
      },
      "id": 2,
      "name": "PragmaDirective",
      "src": "24:33:0"
    },
    {
      "attributes":
      {
        "abstract": false,
        "baseContracts":
        [
          null
        ],
        "contractDependencies":
        [
          null
        ],
        "contractKind": "contract",
        "documentation": null,
        "fullyImplemented": true,
        "linearizedBaseContracts":
        [
          283
        ]

In [10]:
import solcx
solcx.get_solcx_install_folder()

PosixPath('/home/jovyan/.solcx')

In [ ]:
/home/jovyan/.solcx/solc-v0.6.7 # this should be added to path...

## Logic-based / Prolog

In [ ]:
person(arnold). %arbiter
person(lorna). %licensor
person(lisa). %licenscee
person(sunny). % sublicensee
asset(work). % add asset
current_time(D) :-
    get_time(X), stamp_date_time(X, Y, local), date_time_value('date', Y, D).

% set this if removal was requested. though prob not correct as likely that is called each time ...
% maybe this is not possible in prolog but needs some extensions?
% also - list of licenses ?
requested_removal(lorna, lisa, D) :-
    current_time(D).

% limited(license). % or where to 
made_offer(lorna, lisa).
accepts_offer(lisa, lorna).
paid_fee(lisa, lorna, fee).
approves_comments(lorna, lisa).
commissions(lorna, lisa).
%comission_fee(lorna, lisa).

licensor(X) :-
	licenses(X, _).

licensee(Y) :-
	licenses(_, Y).
    
arbiter(A) :-
    assigned_as_arbiter(A, _). % assigned arbiter to license
    
licenses(X, Y, A, D) :- % licensor, licensee, asset, time/license term ( or take license term out again)
    asset(A),
	person(X), 
	person(Y),
	made_offer(X, Y),
	accepts_offer(Y, X),
	paid_fee(Y, X, _),
    current_time(D).

licenses(lorna, lisa, asset, D).
%maybe this should set time...

may_sublicense(Y) :-
    licenses(_, Y, _, _).
    
may_publish(Y) :-
	licenses(X, Y, _, _),
	approves_comments(X,Y).

% Art 2
must_publish(Y) :- % licensee must publish if ...
	licenses(X, Y, _, _), % ... there is a license between X and Y
	approves_comments(X,Y), % ... X approves of the comments Y made
    commission_fee(Y, X), %
	commissions(X,Y). % ... and X commissioned Y to comment.

%Art 3 % deontic logic
forbidden_publish(Y) :- %forbidden to publish if not approved
	not(approves_comments(_, Y)).

forbidden_publish(Y) :- % forbidden to publish if not licensed ( could be combined with above)
    not(licenses(_, Y, _, _)).

must_remove_comments(Y, T) :-
    requested_removal(_, Y, T).
	%licenses(_, Y, _, T2). % which time handling makes sense but is not too convuluted?

breach(Y, T):- % this is not working yet because not sure how to do date arithmetic
    must_remove_comments(Y, T).
	%T is T+24. % fix

% Art 4
breach(SL) :- % breach of L means breach of SL
    breach(_, _). 

terminates(_, Y) :- % this currently treats var names for L and SL differently but does not make sense and should be reduced
    breach(Y, _).

terminates(_, Y) :- 
    breach(Y, _). 

breach_fee(L, _) :-
    breach(L, _).

%breach_fee(Y, _) :-
%    breach(Y, _).
    


The license example was given above. For querying the example, please use the field below.

In [ ]:
?- must_publish(eve).
?- get_time(X), stamp_date_time(X, Y, local).

## ACE / Attempto

Below is an exexcution of the ACE example, with the Ace Parsing Enginge (APE). It is currently read from a file and not from a notebook cell, but the file can be modified and also output of ape changed to other formats. < Link to APE options >

In [8]:
%%ape
An arbiter is a person.
A n:licensor is a person.
A licensee is a person.
A n:sublicensee is a person.
A n:licensor commissions a licensee.
A n:duration-to-remove-comments is 24 hours.
A n:license-fee is an amount.
If there is a n:licensor and there is a licensee then a n:license-fee is required.
If the n:license-fee is paid then the licensee receives a license.
If there is a license then the licensee can v:sublicense.
If there is a license then the licensee can comment.
If the licensee may comment then the n:licensor may approve.
If a n:licensor commissions the licensee then a licensee has to comment.
If the n:licensor does not approve then the licensee may not comment.
If the licensee may not publish then the comments have to be removed within 24 hours.
If the comments have to be removed and are not removed within 24 hours then the license is breached.
If the license is breached then a n:breach-fee has to be paid.

<?xml version="1.0" encoding="UTF-8"?>

<apeResult>
  <duration tokenizer="0.070" parser="0.016" refres="0.001"/>
  <syntax>[[specification,[s,[np,[det,an],[nbar,[n,arbiter]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensor]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensee]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,sublicensee]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensor]]],[vp,[vbar,[vcompl,[v,commissions],[np,[det,a],[nbar,[n,licensee]]]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,'duration-to-remove-comments']]],[vp,[aux,is],[np,[nump,24],[nbar,[n,hours]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,'license-fee']]],[vp,[aux,is],[np,[det,an],[nbar,[n,amount]]]]],'.'],[specification,[cond_s,if,[s_coord,[top_s,[topic,[quant,'there is'],[np,[det,a],[nbar,[n,licensor]]]]],[c

In [9]:
/home/jovyan/bin/ape.bin -file license_from_prolog.ace -solo tokens

[[^,'An',arbiter,is,a,person,'.'],[^,'A',n,:,licensor,is,a,person,'.'],[^,'A',licensee,is,a,person,'.'],[^,'A',n,:,sublicensee,is,a,person,'.'],[^,'A',n,:,licensor,commissions,a,licensee,'.'],[^,'A',n,:,'duration-to-remove-comments',is,24,hours,'.'],[^,'A',n,:,'license-fee',is,an,amount,'.'],[^,'If',there,is,a,n,:,licensor,and,there,is,a,licensee,then,a,n,:,'license-fee',is,required,'.'],[^,'If',the,n,:,'license-fee',is,paid,then,the,licensee,receives,a,license,'.'],[^,'If',there,is,a,license,then,the,licensee,can,v,:,sublicense,'.'],[^,'If',there,is,a,license,then,the,licensee,can,comment,'.'],[^,'If',the,licensee,may,comment,then,the,n,:,licensor,may,approve,'.'],[^,'If',a,n,:,licensor,commissions,the,licensee,then,a,licensee,has,to,comment,'.'],[^,'If',the,n,:,licensor,does,not,approve,then,the,licensee,may,not,comment,'.'],[^,'If',the,licensee,may,not,publish,then,the,comments,have,to,be,removed,within,24,hours,'.'],[^,'If',the,comments,have,to,be,removed,and,are,not,removed,within

In [ ]:
~/bin/ape.bin

In [28]:
/home/jovyan/bin/ape.bin -file license_from_prolog.ace -solo paraphrase

There is an arbiter X1.
The arbiter X1 is a person.

There is a n:licensor X1.
The n:licensor X1 is a person.

There is a licensee X1.
The licensee X1 is a person.

There is a n:sublicensee X1.
The n:sublicensee X1 is a person.

There is a n:licensor X1.
The n:licensor X1 commissions a licensee.

There is a n:duration-to-remove-comments X1.
The n:duration-to-remove-comments X1 is 24 hours.

There is a n:license-fee X1.
The n:license-fee X1 is an amount.

If there is a n:licensor and there is a licensee then a n:license-fee is required.

If a n:license-fee is paid then a licensee receives a license.

If there is a license then there is a licensee X1 and it is possible that the licensee X1 v:sublicense .

If there is a license then there is a licensee X1 and it is possible that the licensee X1 comments.

If there is a licensee X1 and it is admissible that the licensee X1 comments then there is a n:licensor X2 and it is admissible that the n:licensor X2 approves.

If a n:licensor commissi

## Lexon

In [31]:
%%lexonv2
LEX: Evaluation License System.
LEXON: 0.2.12
PREAMBLE: This is a licensing contract for a software evaluation.
 
TERMS:

"Licensor" is [ a person ].
"Arbiter" is [ a person ].

"License" is this contract.

"Licensing Fee" is [ an amount ].
"Breach Fee" is [ an amount ].

The Licensor appoints the Arbiter,
fixes the Licensing Fee,
and fixes the Breach Fee.

CONTRACTS per Licensee:

"Description of Goods" is [ a text ].
"Licensee" is [ a person ].
"Paid" is [ a binary ].
"Commissioned" is [ a binary ].
"Comment Text" is [ a text ].
"Published" is [ a binary ].
"Permission to Comment" is [ a binary ].

The Licensor fixes the Description of Goods.

CLAUSE: License.
The Licensee pays the Licensing Fee to the Licensor,
and the Breach Fee into escrow.
The License is therefore Paid.

CLAUSE: Commission.
The Licensor may certify this License as Commissioned.

CLAUSE: Comment.
The Licensee may register a Comment Text.

CLAUSE: Publication.
The Licensee may certify the License as Published.

CLAUSE: Grant Permission to Comment.
The Licensee may grant the Permission to Comment.

CLAUSE: Examine for Breach.
The Arbiter may Declare Breach, if:
the License is Commissioned and no Comment Text is set;
or the License is Published and there is no Permission to Comment.

CLAUSE: Declare Breach.
The Arbiter may pay the Breach Fee to the Licensor, 
and afterwards terminate this License.



thread 'main' panicked at 'cannot read file: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/main.rs:33:56
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


CalledProcessError: Command 'b'LEX: Evaluation License System.\nLEXON: 0.2.12\nPREAMBLE: This is a licensing contract for a software evaluation.\n \nTERMS:\n\n"Licensor" is [ a person ].\n"Arbiter" is [ a person ].\n\n"License" is this contract.\n\n"Licensing Fee" is [ an amount ].\n"Breach Fee" is [ an amount ].\n\nThe Licensor appoints the Arbiter,\nfixes the Licensing Fee,\nand fixes the Breach Fee.\n\nCONTRACTS per Licensee:\n\n"Description of Goods" is [ a text ].\n"Licensee" is [ a person ].\n"Paid" is [ a binary ].\n"Commissioned" is [ a binary ].\n"Comment Text" is [ a text ].\n"Published" is [ a binary ].\n"Permission to Comment" is [ a binary ].\n\nThe Licensor fixes the Description of Goods.\n\nCLAUSE: Pay.\nThe Licensee pays the Licensing Fee to the Licensor,\nand the Breach Fee into escrow.\nThe License is therefore Paid.\n \nCLAUSE: Commission.\nThe Licensor may certify this License as Commissioned.\n\nCLAUSE: Comment.\nThe Licensee may register a Comment Text.\n\nCLAUSE:

In [27]:
%%lexonv3
LEX: Evaluation License System.
LEXON: 0.2.12
PREAMBLE: This is a licensing contract for a software evaluation.
 
TERMS:

"Licensor" is [ a person ].
"Arbiter" is [ a person ].
"License" is this contract.
"Licensing Fee" is [ an amount ].
"Breach Fee" is [ an amount ].

The Licensor appoints the Arbiter,
fixes the Licensing Fee,
and fixes the Breach Fee.

CONTRACTS per Licensee:

"Description of Goods" is [ a text ].
"Licensee" is [ a person ].
"Paid" is [ a binary ].
"Commissioned" is [ a binary ].
"Comment Text" is [ a text ].
"Published" is [ a binary ].
"Permission to Comment" is [ a binary ].

The Licensor fixes the Description of Goods.

CLAUSE: Pay.
The Licensee pays the Licensing Fee to the Licensor,
and the Breach Fee into escrow.
The License is therefore Paid.

CLAUSE: Commission.
The Licensor may certify this License as Commissioned.

CLAUSE: Comment.
The Licensee may register a Comment Text.

CLAUSE: Publication.
The Licensee may certify the License as Published.

CLAUSE: Grant Permission to Comment.
The Licensee may grant the Permission to Comment.

CLAUSE: Examine for Breach.
The Arbiter may Declare Breach, if:
the License is Commissioned and no Comment Text is set;
or the License is Published and there is no Permission to Comment.

CLAUSE: Declare Breach.
The Arbiter may pay the Breach Fee to the Licensor, 
and afterwards terminate this License.



Lexon: syntax error at line 57: unexpected character. Can't process '[' (ascii 91).
>> stdin  7:   "«Licensor»" is [


CalledProcessError: Command 'b'LEX: Evaluation License System.\nLEXON: 0.2.12\nPREAMBLE: This is a licensing contract for a software evaluation.\n \nTERMS:\n\n"Licensor" is [ a person ].\n"Arbiter" is [ a person ].\n\n"License" is this contract.\n\n"Licensing Fee" is [ an amount ].\n"Breach Fee" is [ an amount ].\n\nThe Licensor appoints the Arbiter,\nfixes the Licensing Fee,\nand fixes the Breach Fee.\n\nCONTRACTS per Licensee:\n\n"Description of Goods" is [ a text ].\n"Licensee" is [ a person ].\n"Paid" is [ a binary ].\n"Commissioned" is [ a binary ].\n"Comment Text" is [ a text ].\n"Published" is [ a binary ].\n"Permission to Comment" is [ a binary ].\n\nThe Licensor fixes the Description of Goods.\n\nCLAUSE: Pay.\nThe Licensee pays the Licensing Fee to the Licensor,\nand the Breach Fee into escrow.\nThe License is therefore Paid.\n \nCLAUSE: Commission.\nThe Licensor may certify this License as Commissioned.\n\nCLAUSE: Comment.\nThe Licensee may register a Comment Text.\n\nCLAUSE:

In [3]:
~/bin/lexon-rust license.lex

error: Error { variant: ParsingError { positives: [name], negatives: [] }, location: Pos(3), line_col: Pos((1, 4)), path: None, line: "LEX: Evaluation License System.␊", continued_line: None }



\begin{equation*}
P(E)   = {n \choose k} p^k (1-p)^{ n-k}
\end{equation*}

Hvitved Cl

s ::= letrec {fi(⃗xi)⟨⃗yi⟩ = ci}ni=1 in c starting τ
c ::= fulfilment
| ⟨e1⟩ k(⃗x) where e2 due d remaining z then c
| if k(⃗x) where e due d remaining z then c1 else c2 | if e then c1 else c2
| c1 and c2
| c1 or c2
| f(⃗e1)⟨⃗e2⟩
e ::=x|v|¬e|e1⋆e2 |e1 ≺e2
(CSL specification)
(no obligations) (obligation) (external choice) (internal choice) (conjunction) (disjunction) (instantiation)
(expression)
(deadline expression)
d ::= after e1 within e2
range over variables, k ∈ K ranges over action kinds, and v ∈
values. Furthermore, ⋆ ∈ {+, −, ×, /, ∧} and ≺∈ {<, =}.

In [15]:
~/bin/lexon < license.lex

Lexon: syntax error at line 56: unexpected character. Can't process '[' (ascii 91).
>> stdin  7:   "«Licensor»" is [

